# FIT5149 S2 2019 Ass2 Report                                 
#### Group No. 34

### SCIENTIFIC DOCUMENT CLASSIFICATION



>   Vaibhavi Bhardwaj - 30154987

>  Hassaan Ali Khan - 30703700

>  Gayatri Aniruddha - 30945305


## Libraries

In [ ]:
import numpy as np 
import pandas as pd
import re
import os
import matplotlib.pylab as plt

%matplotlib inline
import math  
import glob
from numpy import mean
from numpy import std
from matplotlib import pyplot

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.svm import LinearSVC
from sklearn import model_selection, naive_bayes, svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from scipy.sparse import hstack


## Read Train and Test dataset

In [ ]:
# Loading the dataset needed to tarin and test the data


train = pd.read_csv('../train_data_labels.csv')
test = pd.read_csv('../test_data.csv')

# dropping the null rows 
train['abstract'].dropna(inplace=True)



In [ ]:
# loading train and test abstract  
train_text = train['abstract']
test_text = test['abstract']

## Preprocessing of Data

In [ ]:
# Code for preprocessing and lemmitization
# function to take data with lenght greater than or equal to 3 
# lowering the word characters 
def tokenizeRawData(word):
    tokenizer_rawdata = RegexpTokenizer(r'\w{3,}')
    word = tokenizer_rawdata.tokenize(word.lower())
    return (word)


# making a list of english stop words to be removed 
stopwords_list = stopwords.words('english')

# function to remove stop words for a list 
# list to be changed is word_list
def stopwords_function(word_list):
    # removing stop words from the list
    new_list_without_stopwords = [w for w in word_list if w not in stopwords_list]
    # returning the list to be preinted 
    return new_list_without_stopwords      


# class to clean and tokensie the list 
# the words in the list will be cleaned 
class LemmaTokenizer(object):
    def __init__(self):
        # calling the word lemmatizer
        self.wnl=WordNetLemmatizer()
 
    def __call__(self,document):
        
        # calling the tokenize to tokenise greater than length 2 words 
        document = tokenizeRawData(document)
        # removing stop words from the list 
        document = stopwords_function(document)
        # calling 
        return [self.wnl.lemmatize(t) for t in document]

#Reference
# https://stackoverflow.com/questions/47423854/sklearn-adding-lemmatizer-to-countvectorizer

## Defining TFID Vectroizer

In [ ]:
#TFid vectroizer used to extract features of and auto preprocesses data for us
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True, 
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{3,}',
    ngram_range=(1, 3), # unigrams to trigrams 
    min_df=0.01,
    max_df=0.99,
    max_features=50000, # number of max features 
    tokenizer = LemmaTokenizer() # word tokenizer 
    )


## Splitting Data 

In [ ]:
# Splitting data for test and tarin
# 30 percent of the data is for validation
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(train['abstract'],
                                                                    train['label'],
                                                                    test_size=0.3,
                                                                    random_state=0,
                                                                    stratify=train['label'])

## Fitting to Vectroizer

In [ ]:
# fitting the data to the word vectorizer
word_vectorizer.fit(train_text)

# transforming the data for test validation and train 
train_word_features = word_vectorizer.transform(Train_X)
test_x_word_features = word_vectorizer.transform(Test_X)

# tranforming for test data 
test_word_features = word_vectorizer.transform(test_text)

# traning making a stack for training , test and validation test
train_features = hstack([train_word_features])
test_x_features = hstack([test_x_word_features])
test_features = hstack([test_word_features])

scores = []
submission = pd.DataFrame.from_dict({'test_id': test['test_id']})

## Encoding Labels

In [ ]:

# calling the label encoder to assign interger values to 1 to 100 to the labels
le = LabelEncoder()
# fitting the training data 
Train_Y_encode = le.fit_transform(Train_Y)
# tranforming the test lables 
Test_Y_encode = le.fit_transform(Test_Y)


## Cross Validation and Score Prediction on Validation dataset 

In [ ]:
# evaluating dataset 
def evaluating_model(X_train, y_train, X_test, y_test, model):
    
    # define evaluation procedure
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
    
    # evaluate model
    scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
    
    model.fit(X_train, y_train)
    
    predictions = models[i].predict(X_test)
    
    accuracy = accuracy_score(predictions, y_test)
    
    return scores,accuracy,predictions
 

# the final model for evaluation     
def get_modeling():
    model, name = list(), list()
    
    # running LR as that's the final model
    model.append(LogisticRegression(solver='saga', max_iter=1000))
    name.append('LR-saga')
        
    return model, name
 
    
# calling get modling for the 
models, names = get_modeling()

# evaluate each model
for i in range(len(models)):
    # evaluate the model and store results
    scores, accuracy, prediction = evaluating_model(train_features, Train_Y_encode, test_x_word_features, Test_Y_encode, models[i])
    
    # summarize performance
    print('>%s Train Score: %.3f (%.3f) Test Score: %.3f (%.3f)' % (names[i], mean(scores), std(scores), mean(accuracy), std(accuracy)))
    


## Prediction of Complete Train Dataset on Test set 

In [ ]:

# making a model for prediction
model = LogisticRegression(solver='saga', max_iter=1000) 

# Defining features on train
train_word_features1 = word_vectorizer.transform(train['abstract'])
train_features1 = hstack([train_word_features1])

Train_encode = le.fit_transform(train['label'])


# fitting a model for training and testing 
model.fit(train_features1, Train_encode)
# predicting the values 
y_preds = model.predict(test_features)


## Converting to CSV Format

In [ ]:
# converting labels to final values 
label = le.inverse_transform(y_preds)

# ASSIGNING the dataset to a dummy dataframe 
test1 = test 

# removing abstrtact from the dummy dataset
test1 = test1.drop(columns=['abstract'])
# assigning a column 'label'
test1['label'] = 'null'

# making the final data with test results
# test_id, label as the result for the dataset
for index, row in test1.iterrows():
    test1.iloc[index,test1.columns.get_loc('label')] = label[index]
    
# writing to the csv file 
test1.to_csv('pred_labels.csv', index=False)